In [ ]:
import os
import sys
import pickle
import datetime, time
import random
from importlib import reload
from math import sqrt
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

module_paths = {os.pardir}  # package directory
sys.path.extend(module_paths.difference(sys.path))


from nytf import utils, utils2, main_pipeline

pd.options.display.max_columns = 100

In [ ]:
train = pd.read_pickle('train2.pkl')
test = pd.read_pickle('test2.pkl')

In [ ]:
%%time
holidays_features = main_pipeline.HolidayFeaturesExtractor(date_col="pickup_datetime", interest_col="fare_amount", state="NY")
new_train = holidays_features.fit_transform(train)
new_train.index = train.index
train = pd.concat((train,new_train), axis=1)
test = pd.concat((test, holidays_features.transform(test)), axis=1)

In [ ]:
%%time
business_features = main_pipeline.BusinessFeatures(hour_col='hour')
train = pd.concat((train, business_features.fit_transform(train)), axis=1)
test = pd.concat((test, business_features.transform(test)), axis=1)

In [ ]:
def dist(lat1, lon1, lat2, lon2):
    R = 6373000

    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    return R * c

In [ ]:
%%time
train['latitude_diff'] = train['dropoff_latitude'].values - train['pickup_latitude'].values
train['longitude_diff'] = train['dropoff_longitude'].values - train['pickup_longitude'].values
test['latitude_diff'] = test['dropoff_latitude'].values - test['pickup_latitude'].values
test['longitude_diff'] = test['dropoff_longitude'].values - test['pickup_longitude'].values
train['distance2'] = dist(train['pickup_latitude'].values, train['pickup_longitude'].values, train['dropoff_latitude'].values, train['dropoff_longitude'].values)
test['distance2'] = dist(test['pickup_latitude'].values, test['pickup_longitude'].values, test['dropoff_latitude'].values, test['dropoff_longitude'].values)
train['distance2_jfk'] = dist(train['pickup_latitude'].values, train['pickup_longitude'].values, 40.6441666667, -73.7822222222).clip(1000, 10000) - 1000
test['distance2_jfk'] = dist(test['pickup_latitude'].values, test['pickup_longitude'].values, 40.6441666667, -73.7822222222).clip(1000, 10000) - 1000
train['distance2_guardia'] = dist(train['pickup_latitude'].values, train['pickup_longitude'].values, 40.7769271, -73.87396590000003).clip(1400, 2500) - 1400
test['distance2_guardia'] = dist(test['pickup_latitude'].values, test['pickup_longitude'].values, 40.7769271, -73.87396590000003).clip(1400, 2500) - 1400
train['distance2_jfk_dropoff'] = dist(train['dropoff_latitude'].values, train['dropoff_longitude'].values, 40.6441666667, -73.7822222222).clip(1000, 10000) - 1000
test['distance2_jfk_dropoff'] = dist(test['dropoff_latitude'].values, test['dropoff_longitude'].values, 40.6441666667, -73.7822222222).clip(1000, 10000) - 1000
train['distance2_guardia_dropoff'] = dist(train['dropoff_latitude'].values, train['dropoff_longitude'].values, 40.7769271, -73.87396590000003).clip(1400, 2500) - 1400
test['distance2_guardia_dropoff'] = dist(test['dropoff_latitude'].values, test['dropoff_longitude'].values, 40.7769271, -73.87396590000003).clip(1400, 2500) - 1400

In [ ]:
train.memory_usage(deep=True).sum() / 10**9

In [ ]:
train.to_pickle('../../data/processing/train_total_features.pkl')

In [ ]:
test.to_pickle('../../data/processing/test_total_features.pkl')